### Subqueries & EXISTS Operator: Demonstrate advanced SQL logic using subqueries and EXISTS clauses.

### 1. List customer names and states that ordered products with  “Natural Ash” finish and are located outside California.

In [ ]:
SELECT DISTINCT CustomerName, CustomerState
FROM Customer_T CT 
INNER JOIN Order_T OT ON CT.CustomerID = OT.CustomerID
INNER JOIN OrderLine_T OLT ON OT.OrderID = OLT.OrderID
WHERE EXISTS (
	SELECT *
	FROM Product_T
	WHERE ProductID = OLT.ProductID                  --'that ordered products with...' implies a subquery
	AND ProductFinish = 'Natural Ash'				 --since productfinish is not part of the display
	)												 --We want to filter each record by its association w/ 'natural ash' in order
AND CustomerState != 'CA'; --alt: NOT IN ('CA')

### 2. What are the customer states and their order IDs that include product(s) with more than the average standard price? (EXISTS)

In [ ]:
SELECT DISTINCT CustomerState, OT.OrderID
FROM Customer_T CT 
INNER JOIN Order_T OT ON CT.CustomerID = OT.CustomerID
INNER JOIN OrderLine_T OLT ON OT.OrderID = OLT.OrderID
WHERE EXISTS (                                       --'that include.. with...' implies a compound subquery
	SELECT *
	FROM Product_T
	WHERE ProductID = OLT.ProductID
	AND ProductStandardPrice > (
		SELECT AVG(ProductStandardPrice)
		FROM Product_T
		)
	)
;

### 3. Retrieve name and IDs of customers that have not placed any orders or they are located in Texas. Sort the result based on the customer ID

In [ ]:
SELECT CustomerName, CustomerID --DISTINCT NOT needed
FROM Customer_T CT
WHERE CustomerID NOT IN (	    --We can compare the records with a group returned after the subquery finishes
	SELECT CustomerID
	FROM Order_T OT
	)
OR CustomerState = 'TX'
ORDER BY CustomerID;

### 4. Show description and total quantity of products that have been ordered a total of more than 3 quantity and are more expensive than the average price of product lines 1 and 2

In [ ]:
SELECT ProductDescription, SUM(OrderedQuantity) AS "How Many Sold"
FROM OrderLine_T OLT INNER JOIN Product_T PT
ON OLT.ProductID = PT.ProductID
WHERE EXISTS (								--Compound subquery of both product price > avg(price)
	SELECT *								--of productlines 1 & 2
	FROM Product_T
	WHERE ProductID = PT.ProductID
	AND ProductStandardPrice > (			--To compare AVG price per EACH record (of productline 1,2) we must evaluate at this
		SELECT AVG(ProductStandardPrice)	--level first
		FROM Product_T 
		WHERE ProductLineID IN (1,2)
		)
	)
GROUP BY ProductDescription
HAVING "How Many Sold" > 3;